In [4]:
import csv
import numpy as np
import pandas as pd
from pprint import pprint
import pymc as pm
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
pre_df = pd.read_csv('Top_Traversals_demo-1daybehavior_20140401.csv', header=1)

## Data Formatting

In [49]:
pre_df.head()

,,,,,,2014-04-01
Rank,Sessions,Total Sessions,% of Total Sessions,Paths,% of Total Paths,Path
1,792522,4802851,0.16501074049559314,792522,0.16501074049559314,journey.entry->journey.exit
2,413008,4802851,0.08599225751537992,413008,0.08599225751537992,journey.entry->web.entry->webevent.login->webe...
3,283578,4802851,0.05904368051392808,283578,0.05904368051392808,journey.entry->web.entry->webevent.login->webe...
4,241868,4802851,0.05035925536728081,241868,0.05035925536728081,journey.entry->web.entry->webevent.login->webe...


In [36]:
pre_df.describe()

,2014-04-01
count,438983
unique,438983
top,journey.entry->ivr.entry->ivr.proactive balanc...
freq,1


In [85]:
paths = np.array([ 'Path'])
for row_ind in range(2, len(pre_df)-400000):
    path = list(str(pre_df.iloc[row_ind, :]).split())[1]
    paths = np.vstack((paths, path))

print paths

[['Path']
 ['journey.entry->web.entry->webevent.login->webe...']
 ['journey.entry->web.entry->webevent.login->webe...']
 ..., 
 ['journey.entry->mobile.entry->mobileevent.login...']
 ['journey.entry->mobile.entry->mobileevent.login...']
 ['journey.entry->mobile.entry->mobileevent.login...']]


In [86]:
paths = paths[1:]
for journey in range(len(paths)):
    paths[journey] = paths[journey][0].replace('->', ' ')

print paths

[['journey.entry web.entry webevent.login webe...']
 ['journey.entry web.entry webevent.login webe...']
 ['journey.entry web.entry webevent.login webe...']
 ..., 
 ['journey.entry mobile.entry mobileevent.login...']
 ['journey.entry mobile.entry mobileevent.login...']
 ['journey.entry mobile.entry mobileevent.login...']]


In [87]:
paths = np.transpose(paths)
print paths

[['journey.entry web.entry webevent.login webe...'
  'journey.entry web.entry webevent.login webe...'
  'journey.entry web.entry webevent.login webe...' ...,
  'journey.entry mobile.entry mobileevent.login...'
  'journey.entry mobile.entry mobileevent.login...'
  'journey.entry mobile.entry mobileevent.login...']]


In [99]:
Wd =  [len(paths[0][i].split()) for i in range(len(paths[0]))]
print Wd

[4, 4, 4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 2, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 3, 3, 2, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 4, 3, 4, 2, 4, 3, 4, 3, 4, 4, 4, 2, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 2, 4, 4, 4, 2, 4, 5, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 4, 3, 4, 4, 4, 2, 3, 4, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 2, 4, 3, 4, 3, 3, 4, 2, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 3, 4, 3, 4, 4, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 3, 4, 3, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 5, 3, 4, 3, 4, 2, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 3, 4, 3, 3, 4, 4, 4, 2, 4, 4, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 3, 4, 4, 2, 4, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 2, 4, 3, 4, 4, 4, 4, 3, 4, 4, 3, 3, 2, 4, 3, 3, 4, 3, 4, 4, 

## Building Tfidf Matrix

In [90]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(paths[0])
vectors.shape

(38981, 177)

In [107]:
print vectors.idf_

AttributeError: idf_ not found

In [104]:
# V is the vocab or num of words and D is the number of documents
D, V = 38981, 177
# K is the number of topics
K = 10

alpha = np.ones(K)
beta = np.ones(V)

theta = pm.Container([pm.CompletedDirichlet("theta_%s" % i, pm.Dirichlet("ptheta_%s" % i, theta=alpha))\
                      for i in range(D)])
phi = pm.Container([pm.CompletedDirichlet("phi_%s" % k, pm.Dirichlet("pphi_%s" % k, theta=beta))\
                    for k in range(K)])

#Wd commented out because it is defined above
#Wd = [len(doc) for doc in vectors]

z = pm.Container([pm.Categorical('z_%i' % d,
                     p = theta[d],
                     size=Wd[d],
                     value=np.random.randint(K, size=Wd[d]))
                  for d in range(D)])

# cannot use p=phi[z[d][i]] here since phi is an ordinary list while z[d][i] is stochastic                                                                     
w = pm.Container([pm.Categorical("w_%i_%i" % (d,i),
                    p = pm.Lambda('phi_z_%i_%i' % (d,i),
                              lambda z=z[d][i], phi=phi: phi[z]),
                    value=vectors[d][i],
                    observed=True)
                  for d in range(D) for i in range(Wd[d])])

model = pm.Model([theta, phi, z, w])

M = pm.MCMC(model)
M.sample(5000,burn=500)
pm.Matplot.plot(M)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]